In [1]:
import requests
from io import BytesIO
from cachecontrol import CacheControl
from cachecontrol.caches.file_cache import FileCache
from cachecontrol.heuristics import LastModified
from rdflib import Graph, URIRef, Literal, RDF, RDFS, Namespace

session = CacheControl(requests.Session(),
                       cache=FileCache('.cache'),
                       heuristic=LastModified())

g = Graph()
ORG = Namespace('http://www.w3.org/ns/org#')
g.bind('org', ORG)
g.bind('gov', URIRef('https://www.gov.uk/government/organisations/'))

org_url = 'https://www.gov.uk/api/organisations'

while True:
    results = session.get(org_url).json()
    for org_info in results['results']:
        org = URIRef(org_info['web_url'])
        g.add((org, RDF.type, ORG.Organization))
        g.add((org, RDFS.label, Literal(org_info['title'], 'en')))
        if 'parent_organisations' in org_info:
            for parent_org in org_info['parent_organisations']:
                g.add((org, ORG.subOrganizationOf, URIRef(parent_org['web_url'])))
    if 'next_page_url' in results:
        org_url = results['next_page_url']
    else:
        break

print(g.serialize(format='turtle').decode('utf-8')[:10000])
with open('gov_orgs.ttl', 'wb') as f:
    f.write(g.serialize(format='turtle'))

@prefix gov: <https://www.gov.uk/government/organisations/> .
@prefix org: <http://www.w3.org/ns/org#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

gov:academy-for-justice-commissioning a org:Organization ;
    rdfs:label "Academy for Justice Commissioning"@en ;
    org:subOrganizationOf gov:ministry-of-justice .

gov:academy-for-social-justice-commissioning a org:Organization ;
    rdfs:label "Academy for Social Justice Commissioning"@en ;
    org:subOrganizationOf gov:ministry-of-justice .

gov:acas a org:Organization ;
    rdfs:label "Advisory, Conciliation and Arbitration Service"@en ;
    org:subOrganizationOf gov:department-for-business-energy-and-industrial-strategy .

gov:accelerated-access-review a org:Organization ;
    rdfs:label "Accelerated Access Review"@en ;
    org:subOrganizationOf gov:depart